# GPlearn 1 Minute  Period Data Alpha Factor Generating
## What is Alpha Factor?
Alpha (α) is a term used in investing to describe an investment strategy's ability to beat the market, or its "edge." Alpha is thus also often referred to as “excess return” or “abnormal rate of return,” which refers to the idea that markets are efficient, and so there is no way to systematically earn returns that exceed the broad market as a whole. Alpha is often used in conjunction with beta (the Greek letter β), which measures the broad market's overall volatility or risk, known as systematic market risk.  

Alpha is used in finance as a measure of performance, indicating when a strategy, trader, or portfolio manager has managed to beat the market return over some period. Alpha, often considered the active return on an investment, gauges the performance of an investment against a market index or benchmark that is considered to represent the market’s movement as a whole.  

The excess return of an investment relative to the return of a benchmark index is the investment’s alpha. Alpha may be positive or negative and is the result of active investing. Beta, on the other hand, can be earned through passive index investing.(Retrived from https://www.investopedia.com/terms/a/alpha.asp)  

Alpha Factor is a mathmatical expression that tries to identify the strategy that can provide information on the future trend of the market. Alpha factor examples can be found on https://www.spglobal.com/marketintelligence/en/solutions/alpha-factor-library.  
## What is GPlearn and symbolic transformer?
gplearn implements Genetic Programming in Python, with a scikit-learn inspired and compatible API.

While Genetic Programming (GP) can be used to perform a very wide variety of tasks, gplearn is purposefully constrained to solving symbolic regression problems. This is motivated by the scikit-learn ethos, of having powerful estimators that are straight-forward to implement.Genetic programming is capable of taking a series of totally random programs, untrained and unaware of any given target function you might have had in mind, and making them breed, mutate and evolve their way towards the truth.

Think of genetic programming as a stochastic optimization process. Every time an initial population is conceived, and with every selection and evolution step in the process, random individuals from the current generation are selected to undergo random changes in order to enter the next. You can control this randomness by using the random_state parameter of the estimator.  

A symbolic transformer is a supervised transformer that begins by building a population of naive random formulas to represent a relationship. The formulas are represented as tree-like structures with mathematical functions being recursively applied to variables and constants. Each successive generation of programs is then evolved from the one that came before it by selecting the fittest individuals from the population to undergo genetic operations such as crossover, mutation or reproduction. The final population is searched for the fittest individuals with the least correlation to one another.

In [14]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import graphviz

from scipy.stats import rankdata
import scipy.stats as stats

from gplearn import genetic
from gplearn.functions import make_function
from gplearn.genetic import SymbolicTransformer, SymbolicRegressor
from gplearn.fitness import make_fitness

from sklearn.utils import check_random_state
from sklearn.model_selection import train_test_split
import baostock as bs
from ta.volume import VolumeWeightedAveragePrice
import statsmodels.api as sm
from scipy.stats.mstats import zscore
import talib

from scipy.stats import spearmanr
import datetime
from numba import jit
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
import math
import cloudpickle
import matplotlib.pyplot as plt
%matplotlib inline

# Import and Prepare Data

In [15]:
data_df = pd.read_csv('dataset/PTA1分钟历史数据.csv', parse_dates=['date'])

data_df.head(3), data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    8000 non-null   datetime64[ns]
 1   open    8000 non-null   float64       
 2   high    8000 non-null   float64       
 3   low     8000 non-null   float64       
 4   close   8000 non-null   float64       
 5   volume  8000 non-null   float64       
 6   money   8000 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 437.6 KB


(                 date    open    high     low   close  volume       money
 0 2022-06-21 13:55:00  6928.0  6934.0  6926.0  6932.0  2053.0  71143293.0
 1 2022-06-21 13:56:00  6932.0  6934.0  6926.0  6926.0   806.0  27922527.0
 2 2022-06-21 13:57:00  6926.0  6932.0  6926.0  6930.0   678.0  23490440.0,
 None)

In [16]:
fields = ['open','close','high','low','volume','money']
length = []

df = data_df.copy()
df.high = df.high.values.astype('float64')
df.low = df.low.values.astype('float64')
df.close = df.close.values.astype('float64')
df.open = df.open.values.astype('float64')
df.volume = df.volume.values.astype('float64')
df.amount = df.money.values.astype('float64')

df.tail(3), df.head(3), df.isnull().any(), df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    8000 non-null   datetime64[ns]
 1   open    8000 non-null   float64       
 2   high    8000 non-null   float64       
 3   low     8000 non-null   float64       
 4   close   8000 non-null   float64       
 5   volume  8000 non-null   float64       
 6   money   8000 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 437.6 KB


(                    date    open    high     low   close   volume        money
 7997 2022-07-22 14:57:00  5620.0  5640.0  5620.0  5634.0  10553.0  297137303.0
 7998 2022-07-22 14:58:00  5634.0  5640.0  5630.0  5640.0   8149.0  229665983.0
 7999 2022-07-22 14:59:00  5640.0  5658.0  5640.0  5648.0  13682.0  386425287.0,
                  date    open    high     low   close  volume       money
 0 2022-06-21 13:55:00  6928.0  6934.0  6926.0  6932.0  2053.0  71143293.0
 1 2022-06-21 13:56:00  6932.0  6934.0  6926.0  6926.0   806.0  27922527.0
 2 2022-06-21 13:57:00  6926.0  6932.0  6926.0  6930.0   678.0  23490440.0,
 date      False
 open      False
 high      False
 low       False
 close     False
 volume    False
 money     False
 dtype: bool,
 None)

## Calculate The 1 Minute Rate of Return for Backtest

In [17]:
df['1_min_return'] = df.open.pct_change(1).shift(-1)
df = df.dropna()

df.head(5)

,date,open,high,low,close,volume,money,1_min_return
0,2022-06-21 13:55:00,6928.0,6934.0,6926.0,6932.0,2053.0,71143293.0,0.000577
1,2022-06-21 13:56:00,6932.0,6934.0,6926.0,6926.0,806.0,27922527.0,-0.000866
2,2022-06-21 13:57:00,6926.0,6932.0,6926.0,6930.0,678.0,23490440.0,0.000578
3,2022-06-21 13:58:00,6930.0,6934.0,6930.0,6932.0,706.0,24469960.0,0.000289
4,2022-06-21 13:59:00,6932.0,6944.0,6932.0,6936.0,4663.0,161743927.0,0.000577


## Split Training and Testing Datase# 分割训练和测试数据

In [ ]:
from datetime import datetime

# 训练集占比75%，测试集占比25%
train_test_split_date = datetime.strptime('2022-7-14', "%Y-%m-%d")

In [ ]:
train_data=df[(df.date<train_test_split_date)]
train_data=train_data.reset_index(drop=True)

train_data.head(3), train_data.tail(3), train_data.shape

In [ ]:
test_data=df[(df.date>=train_test_split_date)]
test_data=test_data.reset_index(drop=True)

test_data.head(3), test_data.shape, test_data.index

In [ ]:
X_train = train_data.drop(columns=['date','1_min_return']).to_numpy()
y_train = train_data['1_min_return'].values

X_train, X_train.shape, y_train, y_train.shape

In [ ]:
X_test = test_data.drop(columns=['date', '1_min_return']).to_numpy()
y_test = test_data['1_min_return'].values

X_test, X_test.shape, y_test, y_test.shape

# Add More Functions and a Customized Backtest Fitness Function for GPlearn


In [ ]:
init_function = ['add', 'sub', 'mul', 'div','sqrt', 'log','inv','sin','max','min']
def _ts_beta(x1, x2, n):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n[0] == n[1] and n[1] == n[2]:
                u'''need：(list1,list2,number)  return：number
                前 n 期样本 A 对 B 做回归所得回归系数'''
                list1 = x1.flatten().tolist()
                list2 = x2.flatten().tolist()
                n = int(n[0])
                list1 = np.array(list1[-n-1:]).reshape(-1, 1)
                list2 = np.array(list2[-n-1:]).reshape(-1, 1)
                linreg = LinearRegression()
                model = linreg.fit(list1, list2)
                res = linreg.coef_.tolist()[0]
                res = np.array(res+[0]*(len(x1)-len(linreg.coef_))).flatten()
                return res
            else:
                return np.zeros(x1.shape[0])
        except:
            return np.zeros(x1.shape[0])
        
def _ts_resid(x1, x2, n):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n[0] == n[1] and n[1] == n[2]:
                u'''need：(list1,list2,number)  return：number
                前 n 期样本 A 对 B 做回归所得的残差'''
                list1 = x1.flatten().tolist()
                list2 = x2.flatten().tolist()
                n = int(n[0])
                list1 = np.array(list1[-n-1:]).reshape(-1, 1)
                list2 = np.array(list2[-n-1:]).reshape(-1, 1)
                linreg = LinearRegression()
                model = linreg.fit(list1, list2)
                res = list(linreg.intercept_)
                res = np.array(res+[0]*(len(x1)-len([linreg.intercept_]))).flatten()
                return res
            else:
                return np.zeros(x1.shape[0])
        except:
            return np.zeros(x1.shape[0])
        
def _corr(data1,data2,n):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n[0] == n[1] and n[1] ==n[2]:
                window  = n[0]
                x1 = pd.Series(data1.flatten())
                x2 = pd.Series(data2.flatten())
                df = pd.concat([x1,x2],axis=1)
                temp = pd.Series()
                for i in range(len(df)):
                    if i<=window-2:
                        temp[str(i)] = np.nan
                    else:
                        df2 = df.iloc[i-window+1:i,:]
                        temp[str(i)] = df2.corr('spearman').iloc[1,0]
                return np.nan_to_num(temp)
            else:
                return np.zeros(data1.shape[0])
        except:
            return np.zeros(data1.shape[0])
        
def SEQUENCE(n):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n[0] == n[1] and n[1] == n[2]:
                a = n[0]
                res = np.array(list(range(1, a+1)))
                res = res.tolist()+[0]*(len(n)-len(res))
                return np.array(res).flatten()
            else:
                return np.zeros(n.shape[0])
        except:
            return np.zeros(n.shape[0])

def _ts_prod(x, n):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n[0] == n[1] and n[1] == n[2]:
                x = pd.Series(x.flatten())
                n = n[0]
                res = x.prod(min_count=n)
                res = np.array([res]+[0]*(len(x)-1))
                return res
            else:
                return np.zeros(x.shape[0]) 
        except:
            return np.zeros(data1.shape[0]) 
        
def _ts_cov(x,y,window):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n[0] == n[1] and n1[1] == n[2]:
                x = pd.Series(x.flatten())
                y = pd.Series(y.flatten())
                res = x.rolling(window).cov(y).fillna(0).values
                return res
            else:
                return np.zeros(x.shape[0]) 
        except:
            return np.zeros(x.shape[0])

def _ts_lowday(x1, n1):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n1[0] == n1[1] and n1[1] == n1[2]:
                '''need：(list,number)  return：number
                计算 A 前 n 期时间序列中最小值距离当前时点的间隔'''
                n = int(n1[0])
                x = x1.flatten().tolist()    
                lowday = n-x[-n:].index(min(list(x)[-n:]))-1
                return np.array([lowday]+[0]*(len(x)-1))
            else:
                return np.zeros(x1.shape[0]) 
        except:
            return np.zeros(x1.shape[0]) 
        
def _ts_highday(x1, n1):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n1[0] == n1[1] and n1[1] == n1[2]:
                u'''need：(list,number)  return：number
                计算 A 前 n 期时间序列中最大值距离当前时点的间隔'''
                n = int(n1[0])
                x = x1.flatten().tolist()
                highday = n-x[-n:].index(max(list(x)[-n:]))-1
                return np.array([highday]+[0]*(len(x)-1))
            else:
                return np.zeros(x1.shape[0]) 
        except:
            return np.zeros(x1.shape[0]) 
        
def _ts_adv(x,d):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if d[0] == d[1] and d[1] == d[2]:
                d = int(d[0])
                reslist(map(lambda t: np.mean(x.flatten()[t:t+20]),range(len(x.flatten())-d+1)))
                res = res[numpy.isneginf(res)] = 0
                return np.array(res)
            else:
                return np.zeros(x.shape[0])
        except:
            return np.zeros(x.shape[0])
            
def _ts_wma(x1,d):
        with np.errstate(divide='ignore', invalid='ignore'):
            try:
                if d[0] == d[1] and d[1] == d[2]:
                    d = int(d[0])
                    x1 = pd.Series(x1.flatten())
                    weights = np.arange(1,d+1)
                    wma = x1.rolling(d).apply(lambda prices: np.dot(prices, weights)/weights.sum(), raw=True)
                    wma = wma.fillna(0).values
                    return wma
                else:
                    return np.zeros(x1.shape[0])
            except:
                return np.zeros(x1.shape[0])

def _ts_mean(X, d):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if d[0] == d[1] and d[1] == d[2]:
                X = pd.Series(X.flatten())
                res = X.rolling(window=d[0]).mean()
                res = res.fillna(0).values
                return res
            else:
                return np.zeros(X.shape[0])
        except:
            return np.zeros(X.shape[0])

def _decay_linear(data1,n):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n[0] == n[1] and n[1] == n[2]:
                n = int(n[0])
                w = np.arange(1,n+1)
                w = w[::-1]
                w = np.array(w)/np.sum(w)
                res = list(map(lambda t: np.sum(np.multiply(np.array(data1.flatten()[t:t+n]),w)),range(len(data1.flatten())-n+1)))
                return np.array(res).flatten()
            else:
                return np.zeros(data1.shape[0])
        except:
            return np.zeros(data1.shape[0])

def _abs(x1):
    return abs(x1.flatten())

def _ts_abs(data1, n):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n[0] == n[1] and n[1] == n[2]:
                window = int(n[0])
                x1 = data1.flatten()
                temp1 = x1[-window:]
                temp1 = np.abs(temp1)
                temp2 = x1[:-window]
                result = np.append(temp2,temp1)
                return result
            else:
                return np.zeros(data1.shape[0])
        except:
            return np.zeros(data1.shape[0]) 

def _ts_scale(data1, n):
        with np.errstate(divide='ignore', invalid='ignore'):
            try:
                if n[0] == n[1] and n[1] == n[2]:
                    return (int(n[0])*x1 / np.nansum(np.abs(data1.flatten())))
                else:
                    return np.zeros(data1.shape[0])
            except:
                return np.zeros(data1.shape[0])
            
def _signedpower(data1, n):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n[0] == n[1] and n[1] == n[2]:
                return np.sign(data1.flatten()) * np.power(np.abs(data1.flatten()), int(n[0]))
            else:
                return np.zeros(data1.shape[0])
        except:
            return np.zeros(data1.shape[0]) 

def _ts_delta(data1,n):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n[0] == n[1] and n[1] == n[2]:
                a1 = data1-pd.Series(data1.flatten()).shift(periods=int(n[0]))
                a1 = a1.fillna(0).values
                return a1
            else:
                return np.zeros(data1.shape[0])
        except:
            return np.zeros(data1.shape[0]) 

def _ts_delay(data1,n):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n[0] == n[1] and n[1] == n[2]:
                a1 = pd.Series(data1.flatten())
                a1 = a1.shift(periods=int(n[0]))
                a1 = a1.fillna(0).values
                return a1
            else:
                return np.zeros(data1.shape[0])
        except:
            return np.zeros(data1.shape[0]) 

def _ts_corr(data1,data2,n):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n[0] == n[1] and n[1] ==n[2]:
                window = int(n[0])
                x1 = pd.Series(data1.flatten())
                x2 = pd.Series(data2.flatten())
                df = pd.concat([x1,x2],axis=1)
                temp = pd.Series()
                for i in range(len(df)):
                    if i<=window-2:
                        temp[str(i)] = np.nan
                    else:
                        df2 = df.iloc[i-window+1:i,:]
                        temp[str(i)] = df2.corr('spearman').iloc[1,0]
                return np.nan_to_num(temp)
            else:
                return np.zeros(data1.shape[0])
        except:
            return np.zeros(data1.shape[0]) 

def _ts_sum(data,n):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n[0] == n[1] and n[1] ==n[2]:
                window = int(n[0])
                value = np.array(pd.Series(data.flatten()).rolling(window).sum().tolist())
                value = np.nan_to_num(value)
                return value
            else:
                return np.zeros(data.shape[0])
        except:
            return np.zeros(data.shape[0])
        
def _ts_sma(data,n):
    with np.errstate(divide='ignore', invalid='ignore'):   
        try:
            if n[0] == n[1] and n[1] ==n[2]:
                window = int(n[0])
                value = np.array(pd.Series(data.flatten()).rolling(window).mean().tolist())
                value = np.nan_to_num(value)
                return value
            else:
                return np.zeros(data.shape[0])
        except:
            return np.zeros(data.shape[0])

def _ts_stddev(data,n):   
    with np.errstate(divide='ignore', invalid='ignore'): 
        try:
            if n[0] == n[1] and n[1] ==n[2]:
                window = int(n[0])
                value = np.array(pd.Series(data.flatten()).rolling(window).std().tolist())
                value = np.nan_to_num(value)
                return value
            else:
                return np.zeros(data.shape[0])
        except:
            return np.zeros(data.shape[0])

def _ts_rank(x1,n):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n[0] == n[1] and n[1] ==n[2]:
                n = int(n[0])
                x2 = pd.Series(x1.flatten()[-n:])
                res = x2.rank(pct=True).values
                res = res.tolist()+[0]*(x1.shape[0] - res.shape[0])
                return np.array(res)
            else:
                return np.zeros(x1.shape[0]) 
        except:
            return np.zeros(x1.shape[0]) 
        
def _ts_argmin(data,n):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n[0] == n[1] and n[1] ==n[2]:
                window = int(n[0])
                value = pd.Series(data.flatten()).rolling(window).apply(np.argmin) + 1 
                value = np.nan_to_num(value)
                return value
            else:
                return np.zeros(data.shape[0])  
        except:
            return np.zeros(data.shape[0])  

def _ts_argmax(data,n):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n[0] == n[1] and n[1] ==n[2]:
                window = int(n[0])
                value = pd.Series(data.flatten()).rolling(window).apply(np.argmax) + 1 
                value = np.nan_to_num(value)
                return value
            else:
                return np.zeros(data.shape[0])
        except:
            return np.zeros(data.shape[0]) 
        
def _ts_min(data,n):
    import numpy as np
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n[0] == n[1] and n[1] ==n[2]:
                window = int(n[0])
                value = np.array(pd.Series(data.flatten()).rolling(window).min().tolist())
                value = np.nan_to_num(value)
                return value
            else:
                return np.zeros(data.shape[0])
        except:
            return np.zeros(data.shape[0])

def _ts_max(data,n):
    with np.errstate(divide='ignore', invalid='ignore'):
        try:
            if n[0] == n[1] and n[1] ==n[2]:
                window = int(n[0])
                value = np.array(pd.Series(data.flatten()).rolling(window).max().tolist())
                value = np.nan_to_num(value)
                return value
            else:
                return np.zeros(data.shape[0])
        except:
            return np.zeros(data.shape[0])
@jit
def _beta(x1, x2):
    list1 = x1.flatten().tolist()
    list2 = x2.flatten().tolist()
    list1 = np.array(list1).reshape(-1, 1)
    list2 = np.array(list2).reshape(-1, 1)
    linreg = LinearRegression()
    model = linreg.fit(list1, list2)
    res = linreg.coef_.tolist()[0]
    res = np.array(res+[0]*(len(x1)-len(linreg.coef_))).flatten()
    return res

def _resid(x1, x2):
    list1 = x1.flatten().tolist()
    list2 = x2.flatten().tolist()
    list1 = np.array(list1).reshape(-1, 1)
    list2 = np.array(list2).reshape(-1, 1)
    linreg = LinearRegression()
    model = linreg.fit(list1, list2)
    res = linreg.coef_.tolist()[0]
    res = np.array(res+[0]*(len(x1)-len(linreg.intercept_))).flatten()
    return res 

def _beta(x1, x2):
    list1 = x1.flatten().tolist()
    list2 = x2.flatten().tolist()
    list1 = np.array(list1).reshape(-1, 1)
    list2 = np.array(list2).reshape(-1, 1)
    linreg = LinearRegression()
    model = linreg.fit(list1, list2)
    res = linreg.coef_.tolist()[0]
    res = np.array(res+[0]*(len(x1)-len(linreg.coef_))).flatten()
    return res

def _abs(x1):
    return abs(x1.flatten())

def _rank(x1):
    x1 = pd.Series(x1.flatten())
    return x1.rank(pct=True).values
        
def _cube(data):
    return np.square(data.flatten())*data.flatten()

def _square(data):
    return np.square(data.flatten())

def _ts_argmaxmin(data,n):
    try:
        return _ts_argmax(data,n) - _ts_argmin(data,n)
    except:
        return np.zeros(data.shape[0])

def _my_metric_backtest(y, y_pred, w):
    pred = pd.Series(y_pred.flatten()).fillna(0)
    y = pd.Series(y.flatten()).fillna(0)
    short_profit = 0
    long_profit = 0
    held_long = 0
    held_short = 0
    profit = []
    profit_temp = 0
    
    for i in range(len(pred)):
        current_pred = pred.iloc[i]
        current_return = y.iloc[i]
        #open long
        if current_pred>=0.75 and held_long==0 and held_short==0:
            held_long = 1
            held_short = 0
            long_profit += current_return
                    
        #hold long
        elif current_pred>=0.75 and held_long==1 and held_short==0:
            held_long = 1
            held_short = 0
            long_profit += current_return
            
        #open long and close short
        elif current_pred>=0.75 and held_long==0 and held_short==1:
            #close short and record profit
            held_long = 1
            held_short = 0
            short_profit += (-current_return)
            profit.append(short_profit)
            #open long
            long_profit += current_return
                    
        #open short
        elif current_pred<=-0.75and held_long==0 and held_short==0:
            held_long = 0
            held_short = 1
            short_profit += (-current_return)
            
        #keep short
        elif current_pred<=-0.75 and held_long==0 and held_short==1:
            held_long = 0
            held_short = 1
            short_profit += (-current_return)
  
        #close long and open short
        elif current_pred <=-0.75 and held_long==1 and held_short==0:
            #close long
            held_long = 0
            held_short = 1
            long_profit += current_return
            profit.append(long_profit)
            #open short
            short_profit += (-current_return)
                    
        #closeout long 
        elif current_pred<0.75 and current_pred>-0.75 and held_long==1 and held_short==0:
            held_long = 0
            held_short = 0
            long_profit += current_return
            profit.append(long_profit)      
            
        #closeout short    
        elif current_pred<0.75 and current_pred>-0.75 and held_long==0 and held_short==1:
            held_long = 0
            held_short = 0
            short_profit += (-current_return)
            profit.append(short_profit)
    try:        
        total_return = profit[-1]
    except:
        total_return = 0
        
    result = total_return
    
    del pred
    del y
    del short_profit
    del long_profit
    del held_long
    del held_short
    del profit
    
    return result

## Activate The Customized Fuctions

In [ ]:
ts_beta = make_function(function = _ts_beta, name='ts_beta', arity=3, wrap=False)
beta = make_function(function = _beta, name='beta', arity=2, wrap=False)
ts_resid = make_function(function = _ts_resid, name='ts_resid', arity=3, wrap=False)
resid = make_function(function = _resid, name='resid', arity=2, wrap=False)
corr = make_function(function = _corr, name='corr', arity=3, wrap=False) 
SEQUENCE = make_function(function=SEQUENCE, name='SEQUENCE', arity=1, wrap=False) 
ts_cov = make_function(function=_ts_cov, name='ts_cov', arity=3, wrap=False) 
ts_lowday = make_function(function=_ts_lowday, name='ts_lowday', arity=2, wrap=False) 
ts_highday = make_function(function=_ts_highday, name='ts_highday', arity=2, wrap=False) 
ts_adv = make_function(function=_ts_adv, name='ts_adv', arity=2, wrap=False)  
ts_wma = make_function(function=_ts_wma, name='ts_wma', arity=2, wrap=False)       
ts_mean = make_function(function=_ts_mean, name='ts_mean', arity=2, wrap=False) 
decay_linear = make_function(function=_decay_linear, name='decay_linear', arity=2, wrap=False) 
_abs = make_function(function=_abs, name='_abs', arity=1, wrap=False) 
ts_abs = make_function(function=_ts_abs, name='ts_abs', arity=2, wrap=False) 
rank = make_function(function=_rank, name='rank', arity=1, wrap=False) 
ts_scale = make_function(function=_ts_scale, name='ts_scale', arity=2, wrap=False) 
ts_delta = make_function(function=_ts_delta, name='ts_delta', arity=2, wrap=False) 
ts_delay = make_function(function=_ts_delay, name='ts_delay', arity=2, wrap=False)  
ts_sma = make_function(function=_ts_sma, name='ts_sma', arity=2, wrap=False)     
ts_std = make_function(function=_ts_stddev, name='ts_std', arity=2, wrap=False)     
ts_rank = make_function(function=_ts_rank, name='ts_rank', arity=2, wrap=False)
ts_stddev = make_function(function=_ts_stddev, name='ts_stddev', arity=2, wrap=False)
ts_sum = make_function(function=_ts_sum, name='ts_sum', arity=2, wrap=False)
ts_corr = make_function(function=_ts_corr, name='ts_corr', arity=3, wrap=False)
ts_min = make_function(function=_ts_min, name='ts_min', arity=2, wrap=False)
cube = make_function(function=_cube, name='cube', arity=1, wrap=False)
square = make_function(function=_square, name='square', arity=1, wrap=False)
ts_argmaxmin = make_function(function=_ts_argmaxmin, name='ts_argmaxmin', arity=2, wrap=False)
ts_argmax = make_function(function=_ts_argmax, name='ts_argmax', arity=2)
ts_argmin = make_function(function=_ts_argmin, name='ts_argmin', arity=2, wrap=False)
ts_min = make_function(function=_ts_min, name='ts_min', arity=2, wrap=False)
ts_max = make_function(function=_ts_max, name='ts_max', arity=2, wrap=False)
#decay_linear,ts_corr(replaced by corr),
user_function = [SEQUENCE, ts_cov, ts_lowday, ts_highday, ts_adv, ts_wma, ts_mean, _abs, rank, ts_scale, ts_delta, ts_delay, 
                 ts_sma, ts_std, ts_rank, ts_sum, corr, ts_min, cube, square, ts_argmaxmin, ts_argmax, ts_argmin, ts_min, ts_max, ts_beta,
                beta, ts_resid, resid]

## Activate The Customized Fitness Function

In [ ]:
my_metric = make_fitness(function=_my_metric_backtest, greater_is_better=True)

# Fit The Model

In [ ]:
function_set = init_function + user_function
population_size = 2000
generations = 10
random_state= 5
est_gp = SymbolicTransformer(
                            feature_names=fields, 
                            function_set=function_set,
                            generations=generations,
                            metric=my_metric,
                            population_size=population_size,
                            tournament_size=30, 
                            random_state=random_state,
                            verbose=2, 
                            hall_of_fame=100,
                            parsimony_coefficient=0.0001,
                            p_crossover = 0.4,
                            p_subtree_mutation = 0.01,
                            p_hoist_mutation = 0,
                            p_point_mutation = 0.01,
                            p_point_replace = 0.4,
                            n_jobs = 8)
     
X_train = np.nan_to_num(X_train)
y_train = np.nan_to_num(y_train)
est_gp.fit(X_train, y_train)

## Resulting Alpha Factors

In [ ]:
best_programs = est_gp._best_programs
best_programs_dict = {}

for p in best_programs:
    factor_name = 'alpha_' + str(best_programs.index(p) + 1)
    best_programs_dict[factor_name] = {'fitness':p.fitness_, 'expression':str(p), 'depth':p.depth_, 'length':p.length_}
     
best_programs_dict = pd.DataFrame(best_programs_dict).T
best_programs_dict = best_programs_dict.sort_values(by='fitness')
best_programs_dict

# Save The Model

In [ ]:
filename = 'gplearn_PTA_1min_factors.pkl'
with open(filename, 'wb') as f:
     cloudpickle.dump(est_gp, f)

# Read The Model File

In [ ]:
filename = 'gplearn_PTA_1min_factors.pkl'
with open(filename, 'rb') as f:
    est_gp = cloudpickle.load(f)

In [ ]:
best_programs = est_gp._best_programs
best_programs_dict = {}

for p in best_programs:
    factor_name = 'alpha_' + str(best_programs.index(p) + 1)
    best_programs_dict[factor_name] = {'fitness':p.fitness_, 'expression':str(p), 'depth':p.depth_, 'length':p.length_}
     
best_programs_dict = pd.DataFrame(best_programs_dict).T
best_programs_dict = best_programs_dict.sort_values(by='fitness')
best_programs_dict

# Visualize The Alpha Factor Expressions

In [ ]:
def alpha_factor_graph(num):
    # 打印指定num的表达式图

    factor = best_programs[num-1]
    print(factor)
    print('fitness: {0}, depth: {1}, length: {2}'.format(factor.fitness_, factor.depth_, factor.length_))

    dot_data = factor.export_graphviz()
    graph = graphviz.Source(dot_data)
    graph.render('images/alpha_factor_graph', format='png', cleanup=True)
    
    return graph

graph1 = alpha_factor_graph(1)
graph1

In [ ]:
factor = best_programs[0]
print(factor)
print('fitness: {0}, depth: {1}, length: {2}'.format(factor.fitness_, factor.depth_, factor.length_))

# Alpha Factor Analysis

## 因子回测函数 

In [ ]:
@jit
def _factor_backtest(factor_perd, market_price):
    pred = pd.Series(factor_perd.flatten()).fillna(0)
    evaluation = []
    slippage = 4
    shares = 25
    comission = 0.00025
     
    backtest_data = market_price
    trades = pred

    short_open = 0
    long_open = 0
    held_long = 0
    held_short = 0
    profit = []
    profit_temp = 0
    initial_assets = shares*max(backtest_data.open.values)
    initial_cash = shares*max(backtest_data.open.values)
    net_worth = [initial_cash]
    
    for i in range(len(trades)):
        current_pred = trades.iloc[i]
        current_close = backtest_data.iloc[i].open.astype('float')
        #open long
        if current_pred>=0.75 and held_long==0 and held_short==0:
            held_long = 1
            held_short = 0
            long_open = current_close+slippage
            short_open = 0
            #print('open long')
                    
        #hold long
        elif current_pred>=0.75 and held_long==1 and held_short==0:
            #print('hold long')
            pass
                
            #open long and close short
        elif current_pred>=0.75 and held_long==0 and held_short==1:
            held_long = 1
            #close short and calculate profit
            held_short = 0
            profit_temp = (short_open-(current_close+slippage))*shares*(1-comission)
            initial_cash += profit_temp
            profit.append(profit_temp)
            net_worth.append(initial_cash)
            #open long
            short_open = 0
            long_open = current_close+slippage
            #print('open long and close short')
                    
        #open short
        elif current_pred<=-0.75and held_long==0 and held_short==0:
            held_long = 0
            held_short = 1
            long_open = long_open
            short_open = current_close+slippage
            profit = profit
            #print('open short')
            
        #keep short
        elif current_pred<=-0.75 and held_long==0 and held_short==1:
            #print('keep short')
            pass
                
        #close long and open short
        elif current_pred <=-0.75 and held_long==1 and held_short==0:
            #close long
            held_long = 0
            held_short = 1
            profit_temp = ((current_close-slippage)-long_open)*shares*(1-comission)
            initial_cash += profit_temp
            profit.append(profit_temp)
            net_worth.append(initial_cash)
            #open short
            long_open = 0
            short_open = current_close-slippage
            profit = profit
            #print('close long and open short')
                    
        #closeout long
        elif current_pred<0.75 and current_pred>-0.75 and held_long==1 and held_short==0:
            held_long = 0
            held_short = 0
            profit_temp = ((current_close-slippage)-long_open)*shares*(1-comission)
            initial_cash += profit_temp
            profit.append(profit_temp)
            net_worth.append(initial_cash)
            short_open = 0
            long_open = 0
            #print('closeout long')        
            
        #closeout short    
        elif current_pred<0.75 and current_pred>-0.75 and held_long==0 and held_short==1:
            held_long = 0
            held_short = 0
            profit_temp = (short_open-(current_close+slippage))*shares*(1-comission)
            initial_cash += profit_temp
            profit.append(profit_temp)
            net_worth.append(initial_cash)
            short_open = 0
            long_open = 0
            #print('closeout short')
            
    total_return = (initial_cash-initial_assets)/initial_assets
    print('总收益率', total_return)
    shaprpe_df = pd.Series(profit)
    sharpe_temp = (shaprpe_df - shaprpe_df.shift(1))/shaprpe_df.shift(1)
    sharpe = sharpe_temp.mean()/sharpe_temp.std()*np.sqrt(len(profit))
          
    a = np.maximum.accumulate(net_worth)
    l = np.argmax((np.maximum.accumulate(net_worth) - net_worth)/np.maximum.accumulate(net_worth))
    k = np.argmax(net_worth[:l])
    max_draw = (net_worth[k] - net_worth[l])/(net_worth[l]) 
    print('最大回撤', max_draw)        
        
    win_count = 0
    loss_count = 0
    initial_profit = 0
    for i in range(len(net_worth)):
        current_profit = net_worth[i]
        if i==0:
            if current_profit > initial_assets:
                win_count += 1
            else:
                loss_count += 1
        else:
            last_profit = net_worth[i-1]
            if current_profit > last_profit:
                win_count += 1
            else:
                loss_count += 1
    win_rate = win_count/len(net_worth)
    print('胜率', win_rate)
    
    total_gain = 0
    total_loss = 0
    for i in range(len(profit)):
        if profit[i] >0:
            total_gain += profit[i]
        else:
            total_loss += profit[i]
    gain_loss_ratio = (total_gain/win_count)/(abs(total_loss)/loss_count)
    print('盈亏比', gain_loss_ratio)
    
    result = total_return*np.nan_to_num(sharpe,nan=1)*win_rate*gain_loss_ratio*(1-max_draw)
    
    x = np.array(net_worth).reshape(len(net_worth),)
    y = np.arange(len(net_worth))
    
    plt.rcParams['font.sans-serif'] = ['KaiTi'] # 指定默认字体
    plt.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题
    fig = plt.figure(figsize=(16,9))
    plt.plot(y, x)
    plt.title('因子资金曲线',fontsize=20)
    plt.xlabel('交易次数',fontsize=20)
    plt.ylabel('账户净值',fontsize=20)
    plt.show()

    return result

## Prepare Alpha Factor Backtest Results (In-Sample)

In [ ]:
factors_pred = est_gp.transform(X_train)
factors_pred.shape, X_train.shape

In [ ]:
pred_data = pd.DataFrame(factors_pred).T.T
pred_data, pred_data.iloc[:,[0]]

## Alpha Factor Backtest Results (In-Sample)

In [ ]:
#Total Rate of Return: 总收益率
#Max drawdown: 最大回撤
#Wining Rate: 胜率
#Win-loss ratio:盈亏比
#Graph Title: Factor Backtest Account Networth cruve
#Graph X Axis: Number of Trades
#Graph y Axis: Account Networth
_factor_backtest(pred_data.iloc[:,[0]].values, train_data)

## Prepare Alpha Factor Backtest Results (Out-of-Sample)

In [ ]:
factors_pred_1 = est_gp.transform(X_test)
factors_pred_1.shape, X_test.shape

In [ ]:
pred_data_1 = pd.DataFrame(factors_pred_1).T.T
pred_data_1, pred_data_1.iloc[:,[0]]

## Alpha Factor Backtest Results (Out-of-Sample)

In [ ]:
#Total Rate of Return: 总收益率
#Max drawdown: 最大回撤
#Wining Rate: 胜率
#Win-loss ratio:盈亏比
#Graph Title: Factor Backtest Account Networth cruve
#Graph X Axis: Number of Trades
#Graph y Axis: Account Networth
_factor_backtest(pred_data_1.iloc[:,[0]].values, test_data)